In [4]:
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv
from groq import Groq

_ = load_dotenv(find_dotenv())

In [3]:
from langgraph.graph import StateGraph, END 
from typing import TypedDict, Annotated
import operator # Agent states
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import Groq # langchain wrapper
from langchain_community.tools.tavily_search import TavilySearchResults

ModuleNotFoundError: No module named 'langgraph'

In [10]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

NameError: name 'TavilySearchResults' is not defined

In [1]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
# simple agent: annotated list of messages as above

NameError: name 'TypedDict' is not defined

In [5]:
class Agent:
    # call groq
    # check weather an action present
    # take that action
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_groq)
        graph.add_node("action", self.take_action)
        # check weather an action present
        graph.add_condiontionl_edges(
            "llm", # node where the edge start
            ..., # function 
            {True: "action", False: END} # how to map response of the function for next node
        )
        graph.add_edge("action", "llm") # regular edge action to llm
        graph.set_entry_point("llm")
        self.graph = graph.compile() # create a langchain runnable
        self.tools = {t.name: t for t in tools} 
        self.model = model.bind_tools(tools)

# conditional edge (any tool calls return True)
def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

# llm node (function)
def call_groq(self, state: AgentState):
    messages = state['messages']
    if self.system:
        messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    return {'messages': [message]}

def take_actions(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling: {t}")
        result = self.tools[t['name']].invoke(t['args'])
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name']))
    print("Back to the model!")
    return {'messages': results}

NameError: name 'AgentState' is not defined

In [4]:
# Lang graph
# Describe and Orchestrate the controlflow
    # Cyclic Graphs - remember previous 
    # Presistence - remember previous iterations, have multiple conversations
    # Human in the loop
# Nodes: Agents or functions
# Edges: connect nodes
# Conditional edges: decisions

# Agent State: 

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGroq(model=" ")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
# displaying langgraph
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result

In [ ]:
result['messages'][-1].content